In [1]:
using Plots
using SpecialFunctions
using Reactive, Interact

In [6]:
function target(x)
    return @. e^(-x) * cos(4π * x)
end;

"""

Returns Σ (x - min(X))^2 for x in X

Use case : Will return smaller values for values that have a higher gradient toward the minimum value, without calculating the derivative.
This will still work for small sized samples.

"""
function barcode(x)
    F = min_max(x)[1]
    return sum( ((y - F)^2 for y in x ))
end

function min_max(X)
    m, M = typemax(X[1]), typemin(X[1])
    for x in X
        m, M = x < m ? x : m, x > M ? x : M
    end
    return m, M
end;

"""
Finds the minimum window size for which the variation of a 1-dimensional function (represented by) converges, that is: decreases below epsilon. If Y=f(X) is a measurement then epsilon should be the measurement error.

The window is interpreted as indices in x, so is an integer. If no window is found, -1 is returned.
"""
function tuneWindow(Y, epsilon)
    N = length(Y)
    w = Int(round(N/2))
    index_found, w_found = -1, -1
    limit = Int(round(N / 8))
    while w > limit
        index_found = sliding(Y, w, epsilon)
        if index_found != -1
            w_found = w
        end
        w = Int(round(w/2))
#         w -= 1
    end
    return index_found, w_found
end

function sliding(Y, windowsize, epsilon)
    N = length(Y)
    start = -1
    if windowsize >= N
        return start
    end
    m, M = typemax(Y[1]), typemin(Y[1])
    for i in 1:N-windowsize
        if i == 1
            m, M = min_max(Y[i:i+windowsize])
        else
            old = Y[i-1]
            new = Y[i+windowsize]
            # Checking the entire window again for min max is exponential, we only need this if m or M was the first element of the previous window
            if m == old || M == old
                m, M = min_max(Y[i:i+windowsize])
            else
                m, M = new < m ? new : m, new > M ? new : M
            end
        end
        diff = abs(m - M)
        if diff < epsilon && start == -1
            start = i # Record first position where convergence is suspected
        end
        if diff >= epsilon && start != -1
            start = -1 # Variance is increasing, reset.
        end
    end
    return start
end;

In [ ]:
plotly()
N = 1000
X = linspace(0, 3*π , N)
Y = target(X)
m,M = min_max(Y)
epsilon = 0.1
index, windowsize = tuneWindow(Y, epsilon)
if index != -1
    x = X[index]
    y = Y[index]
    println("First threshold found at index $index where X=$x and Y=$y with windowsize=$windowsize")
    plotly()
    plot(X, Y, label="Dampened Cosine")
    plot!([x for y in Y], X/min_max(X)[2], label="Threshold with window $windowsize")
    plot!(title="Sliding window threshold with ϵ $epsilon")
    plot!(xlabel="X", ylabel="f(x)")
else
    println("No index found!!")
end

In [ ]:
m = readdlm("data.txt")
cell_count, value_count = size(m)
X = 1:value_count
Ys = [ copy(m[i,:]) for i in 1:cell_count];
# results = [(0,0) for i in 1:length(Ys)]
@manipulate for ϵ in reverse(0.005:0.005:0.1)
    results = [(0,0) for i in 1:length(Ys)]
    for (cell_id, Y_i) in enumerate(Ys)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            results[cell_id] = (index, windowsize)
        end
    end
    plotly()
    for (i, y) in enumerate(Ys)
        label = "Cell $i"
        i ==  1 ? plot(X, y, lab=label) : plot!(X, y, lab=label)
    end
    for (i, (index, window)) in enumerate(results)
        if index != 0
            x = X[index]
            plot!([x for _ in 1:value_count], X/min_max(X)[2], label="Threshold for cell $(i) and window $window")
        end
    end
    plot!(title="Variance Threshold for ϵ $ϵ")
    plot!(xlabel="Proximity Threshold (nm)", ylabel="#blinks (%)")
end


In [ ]:
m = readdlm("data.txt")
cell_count, value_count = size(m)
X = 1:value_count
Ys = [[ sum(m[:,i])/cell_count for i in 1:value_count]]
results = [(0,0) for i in 1:length(Ys)]
@manipulate for ϵ in reverse(0.01:0.005:0.1)
    for (cell_id, Y_i) in enumerate(Ys)
        m,M = min_max(Y_i)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            x = X[index]
            y = Y_i[index]
            results[cell_id] = (index, windowsize)
        end
    end
    plotly()
    for (i, y) in enumerate(Ys)
        label = "Average of 9 Cells"
        i ==  1 ? plot(X, y, lab=label) : plot!(X, y, lab=label)
    end
    for (i, (index, window)) in enumerate(results)
        label = "Threshold for cell $i"
        if index != 0
            x = X[index]
            plot!([x for _ in 1:value_count], X/min_max(X)[2], label="Threshold for Average")
        end
    end
    plot!(title="Variance Threshold for ϵ $ϵ")
    plot!(xlabel="Proximity Threshold (nm)", ylabel="#blinks (%)")
end

In [27]:
m = readdlm("data/pct.txt")
p = readdlm("data/total")
cell_count, value_count = size(m)
X = 1:value_count
Yts = [ copy(m[i,:]) for i in 1:cell_count];
Ys = similar(Yts)
for (rowindex, y) in enumerate(Yts)
    Ys[rowindex] = y / p[rowindex]
end

results = [(0,0) for i in 1:length(Ys)]
@manipulate for ϵ in reverse(0.001:0.00001:0.1)
    results = [(0,0) for i in 1:length(Ys)]
    for (cell_id, Y_i) in enumerate(Ys)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            results[cell_id] = (index, windowsize)
        end
    end
    plotly()
    for (i, y) in enumerate(Ys)
#         label = "$(i)0 %"
        label = ""
        i ==  1 ? plot(X, y, lab=label) : plot!(X, y, lab=label)
    end
    for (i, (index, window)) in enumerate(results)
        if index != 0
            x = X[index]
            plot!([x for _ in 1:value_count], (X/min_max(X)[2])/1.5, label="Threshold $x nm @ $(i)0 %")
        end
    end
    plot!(title="Variance Threshold for ϵ $ϵ")
    plot!(xlabel="Proximity Threshold (nm)", ylabel="#blinks degree>avg(%)")
end

Interact.Options{:SelectionSlider,Any}(1: "input" = 0.0505 Any , "ϵ", 0.0505, "0.0505", 4951, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.1"=>0.1,"0.09999"=>0.09999,"0.09998"=>0.09998,"0.09997"=>0.09997,"0.09996"=>0.09996,"0.09995"=>0.09995,"0.09994"=>0.09994,"0.09993"=>0.09993,"0.09992"=>0.09992,"0.09991"=>0.09991…), Dict{Any,Any}(Pair{Any,Any}(0.09855, "0.09855"),Pair{Any,Any}(0.07296, "0.07296"),Pair{Any,Any}(0.07056, "0.07056"),Pair{Any,Any}(0.08293, "0.08293"),Pair{Any,Any}(0.0932, "0.0932"),Pair{Any,Any}(0.00358, "0.00358"),Pair{Any,Any}(0.08986, "0.08986"),Pair{Any,Any}(0.05586, "0.05586"),Pair{Any,Any}(0.08005, "0.08005"),Pair{Any,Any}(0.01046, "0.01046")…)), Any[], Any[], true, "horizontal")

In [ ]:
m = readdlm("./data/data_1.txt")
p = readdlm("./data/total_1.txt")
cell_count, value_count = size(m)
X = 1:value_count
Yts = [ copy(m[i,:]) for i in 1:cell_count];
Ys = similar(Yts)
for (rowindex, y) in enumerate(Yts)
    Ys[rowindex] = y / p[rowindex]
end

tresholds = []
for ϵ in reverse(0.028:0.00001:0.08)
    results = [(0,0) for i in 1:length(Ys)]
    for (pct_id, Y_i) in enumerate(Ys)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            results[pct_id] = (index, windowsize)
        end
    end
    push!(tresholds, (ϵ, results))
end

In [23]:
cellcount = 5
cells = Dict()
for cellindex in 1:cellcount
    m = readdlm("./data/data_$cellcount.txt")
    p = readdlm("./data/total_$cellcount.txt")
    cell_count, value_count = size(m)
    X = 1:value_count
    Yts = [ copy(m[i,:]) for i in 1:cell_count];
    Ys = similar(Yts)
    for (rowindex, y) in enumerate(Yts)
        Ys[rowindex] = y / p[rowindex]
    end

    thresholds = []
    for ϵ in reverse(0.028:0.00001:0.08)
        results = [(0,0) for i in 1:length(Ys)]
        for (pct_id, Y_i) in enumerate(Ys)
            index, windowsize = tuneWindow(Y_i, ϵ)
            if index != -1
                results[pct_id] = (index, windowsize)
            end
        end
        push!(thresholds, (ϵ, results))
    end
    cells[cellindex] = thresholds
end

Observe the variance of the percentile plots. Note that this will obviously decrease as epsilon increases. So variance in this form is not a good metric of what we aim to measure here.

In [29]:
plotly()
for cellindex in 1:cellcount
    thresholds = cells[cellindex]
    vars = []
    es = []
    for (i,(ϵ, result)) in enumerate(thresholds)
    #     println("Results for $ϵ are $result")
        push!(vars, barcode([q[1] for q in result[1:10]]))
        push!(es, ϵ)
    end
    display(plot(es, vars, label="Sample Variance for thresholds over [10-100%] blinks", xlabel="ϵ (% blinks degree>avg degree)"))
    besti = indmin(vars)
    println("Best ϵ for cell $cellindex is $(es[besti])")
end

Best ϵ for cell 1 is 0.05618
Best ϵ for cell 2 is 0.05618
Best ϵ for cell 3 is 0.05618
Best ϵ for cell 4 is 0.05618
Best ϵ for cell 5 is 0.05618


We want to observe what happens with the upper half percentiles, that is, 50-100%, in function of epsilon. In essence we're asking if there is a diminishing return pattern, and if so, for what epsilon does that happen?

In [33]:
cellcount = 5
ϵ = 0.05618
for cellindex in 1:cellcount
    m = readdlm("data/data_$cellindex.txt")
    p = readdlm("data/total_$cellindex.txt")
    cell_count, value_count = size(m)
    X = 1:value_count
    Yts = [ copy(m[i,:]) for i in 1:cell_count];
    Ys = similar(Yts)
    for (rowindex, y) in enumerate(Yts)
        Ys[rowindex] = y / p[rowindex]
    end

    
    results = [(0,0) for i in 1:length(Ys)]
    for (cell_id, Y_i) in enumerate(Ys)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            results[cell_id] = (index, windowsize)
        end
    end
    plotly()
    for (i, y) in enumerate(Ys)
#         label = "$(i)0 %"
        label = ""
        i ==  1 ? plot(X, y, lab=label) : plot!(X, y, lab=label)
    end
    for (i, (index, window)) in enumerate(results)
        if index != 0
            x = X[index]
            plot!([x for _ in 1:value_count], (X/min_max(X)[2])/1.5, label="Threshold $x nm @ $(i)0 %")
        end
    end
    plot!(title="Variance Threshold for ϵ $ϵ")
    display(plot!(xlabel="Proximity Threshold (nm)", ylabel="Normalized #blinks degree > avg()"))
end